In [2]:
#import libraries
import nltk, re,pprint
import numpy as np
import pandas as pd
import requests 
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

import nltk
nltk.download('treebank')

#reading the treebank tagged sentences
wsj=list(nltk.corpus.treebank.tagged_sents())

print(wsj[:40])

random.seed(1234)
train_set, test_set = train_test_split(wsj,test_size=0.3)
print(len(train_set))
print(len(test_set))
print(train_set[:40])

#getting list of tagged words
train_tagged_words=[tup for sent in train_set for tup in sent]
len(train_tagged_words)

#tokens
tokens=[pair[0] for pair in train_tagged_words]
tokens[:10]

#vocabulary
V=set(tokens)
print(len(V))

#number of tags
T=set([pair[1] for pair in train_tagged_words])
len(T)

print(T)

#emission probability
#computing P(w/t) amd starting in T x V matrix
t=len(T)
v=len(V)
w_given_t= np.zeros((t,v))

#compute word given tag: Emission Probablility 
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list =[pair for pair in train_bag if pair[1]==tag]
    count_tag=len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    return (count_w_given_tag, count_tag)

#examples
#large
print("\n", "large")
print(word_given_tag('large', 'JJ'))
print(word_given_tag('large', 'VB'))
print(word_given_tag('large', 'NN'),"\n")
#will
print("\n", "will")
print(word_given_tag('will', 'MD'))
print(word_given_tag('will', 'NN'))
print(word_given_tag('will', 'VB'),"\n")
#BOOK
print("\n", "book")
print(word_given_tag('book', 'NN'))
print(word_given_tag('book', 'VB'))

In [15]:
#transition probability
#compute tag given tag: tag2(t2) given tag1(t1), i.e. Transition Probablity
def t2_given_t1(t2,t1, train_bag=train_tagged_words):
    tags=[pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 =0;
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1]==t2:
            count_t2_t1 +=1
    return (count_t2_t1, count_t1)

In [16]:
#examples
print(t2_given_t1(t2='NNP', t1='JJ'))
print(t2_given_t1('NN','JJ'))
print(t2_given_t1('NN','DT'))
print(t2_given_t1('NNP','VB'))
print(t2_given_t1(',','NNP'))
print(t2_given_t1('PRP','PRP'))
print(t2_given_t1('VBG','NNP'))

(148, 4110)
(1839, 4110)
(2696, 5756)
(55, 1773)
(952, 6493)
(2, 1191)
(5, 6493)


In [17]:
#PLEASE NOTE p(TAG/START) IS SAME AS P(TAG/'.')
print(t2_given_t1('DT','.'))
print(t2_given_t1('VBG','.'))
print(t2_given_t1('NN','.'))
print(t2_given_t1('NNP','.'))

(578, 2715)
(11, 2715)
(117, 2715)
(508, 2715)


In [18]:
#CREATING TXT transition matrix of tags
#each column is t2, each row is t1
# thus M(i,j) represents P(tj given ti)
tags_matrix = np.zeros((len(T),len(T)),dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)):
        tags_matrix[i,j]=t2_given_t1(t2,t1)[0]/t2_given_t1(t2,t1)[1]

In [19]:
tags_matrix

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.04076087, 0.00543478,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.03267974, 0.        ,
        0.        ],
       ...,
       [0.        , 0.00513347, 0.        , ..., 0.07340863, 0.00205339,
        0.        ],
       [0.        , 0.        , 0.0075188 , ..., 0.02255639, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00526316, 0.        ,
        0.        ]], dtype=float32)

In [20]:
tags_df= pd.DataFrame(tags_matrix, columns=list(T), index=list(T))

In [21]:
tags_df

WP$         :        RP        DT        TO        ``         .  \
WP$     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
:       0.000000  0.000000  0.000000  0.130435  0.000000  0.048913  0.010870   
RP      0.000000  0.000000  0.000000  0.202614  0.026144  0.019608  0.026144   
DT      0.000000  0.000000  0.000000  0.000869  0.000347  0.006776  0.001216   
TO      0.000000  0.000000  0.000000  0.121514  0.000000  0.005976  0.000000   
``      0.000000  0.000000  0.000000  0.182731  0.000000  0.000000  0.000000   
.       0.000000  0.002947  0.000000  0.212891  0.000737  0.069245  0.000000   
PRP$    0.000000  0.000000  0.000000  0.000000  0.000000  0.001845  0.000000   
UH      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  1.000000   
MD      0.000000  0.000000  0.000000  0.003091  0.000000  0.004637  0.000000   
VB      0.000564  0.001128  0.024253  0.232939  0.016920  0.006204  0.016356   
NNPS    0.000000  0.044944  0.000000  0.011236  0.000000  0.000000  0.078652   
#       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
-LRB-   0.000000  0.000000  0.000000  0.088889  0.000000  0.000000  0.000000   
-NONE-  0.000000  0.007199  0.001091  0.052792  0.181719  0.002400  0.092714   
IN      0.000143  0.000143  0.000143  0.318020  0.001569  0.006848  0.001855   
NNS     0.000711  0.018018  0.000237  0.013039  0.020389  0.001185  0.119014   
LS      0.000000  0.166667  0.000000  0.000000  0.000000  0.000000  0.333333   
RBR     0.000000  0.010417  0.000000  0.072917  0.000000  0.000000  0.041667   
VBG     0.000000  0.002950  0.017699  0.189774  0.046214  0.003933  0.013766   
NN      0.000108  0.013073  0.000540  0.005078  0.021283  0.002809  0.102312   
VBD     0.000000  0.001388  0.016189  0.131822  0.025439  0.003238  0.006013   
CC      0.000000  0.000647  0.000000  0.109961  0.005821  0.005821  0.000000   
PDT     0.000000  0.000000  0.000000  0.904762  0.000000  0.000000  0.000000   
VBP     0.000000  0.002232  0.011161  0.112723  0.012277  0.005580  0.008929   
CD      0.000389  0.005445  0.000000  0.001167  0.027227  0.000389  0.050175   
WDT     0.000000  0.003257  0.000000  0.029316  0.000000  0.003257  0.000000   
SYM     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
''      0.002088  0.000000  0.000000  0.116910  0.004175  0.027140  0.000000   
VBN     0.000000  0.001388  0.008328  0.062457  0.013879  0.002082  0.008328   
JJR     0.000000  0.007018  0.000000  0.021053  0.010526  0.000000  0.073684   
EX      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
PRP     0.000000  0.002519  0.004198  0.005877  0.011755  0.004198  0.033585   
-RRB-   0.000000  0.145833  0.000000  0.062500  0.020833  0.010417  0.125000   
FW      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
JJS     0.000000  0.007874  0.000000  0.015748  0.000000  0.000000  0.039370   
WP      0.000000  0.000000  0.000000  0.054945  0.000000  0.000000  0.000000   
RBS     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
VBZ     0.000000  0.004689  0.010047  0.141326  0.004019  0.010717  0.002679   
$       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
JJ      0.000000  0.002676  0.000243  0.003650  0.009246  0.002190  0.021655   
,       0.000585  0.000000  0.000000  0.139596  0.002927  0.013755  0.000000   
NNP     0.000000  0.006469  0.000154  0.002310  0.004312  0.000924  0.050516   
RB      0.000000  0.005133  0.000000  0.049795  0.013347  0.002053  0.040554   
WRB     0.000000  0.000000  0.007519  0.323308  0.000000  0.000000  0.000000   
POS     0.000000  0.000000  0.000000  0.000000  0.000000  0.008772  0.012281   

            PRP$        UH        MD  ...        WP       RBS       VBZ  \
WP$     0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
:       0.002717  0.000000  0.013587  ...  0.010870  0.000000  0.010870   
RP      0.052288  0.000000 

In [22]:
tags_df.loc['.', :]

WP$       0.000000
:         0.002947
RP        0.000000
DT        0.212891
TO        0.000737
``        0.069245
.         0.000000
PRP$      0.006630
UH        0.000000
MD        0.000000
VB        0.001105
NNPS      0.001842
#         0.000000
-LRB-     0.005157
-NONE-    0.020626
IN        0.122652
NNS       0.043462
LS        0.001842
RBR       0.001105
VBG       0.004052
NN        0.043094
VBD       0.000368
CC        0.047145
PDT       0.000368
VBP       0.000000
CD        0.006630
WDT       0.000000
SYM       0.000000
''        0.060405
VBN       0.001473
JJR       0.003315
EX        0.002947
PRP       0.056722
-RRB-     0.005525
FW        0.000000
JJS       0.000737
WP        0.002947
RBS       0.000737
VBZ       0.002210
$         0.001473
JJ        0.034254
,         0.000000
NNP       0.187109
RB        0.040884
WRB       0.006998
POS       0.000000
Name: ., dtype: float32

In [23]:
#heatmap of tags matrix
#T(i,j) means P(tag j given tag i)
plt.figure(figsize=(18,12))
sns.heatmap(tags_df)
plt.show()

In [24]:
#frequent tags
#filter the df to get P(t2,t1)>0.5
tags_frequent = tags_df[tags_df>0.5]
plt.figure(figsize=(18,12))
sns.heatmap

<function seaborn.matrix.heatmap(data, *, vmin=None, vmax=None, cmap=None, center=None, robust=False, annot=None, fmt='.2g', annot_kws=None, linewidths=0, linecolor='white', cbar=True, cbar_kws=None, cbar_ax=None, square=False, xticklabels='auto', yticklabels='auto', mask=None, ax=None, **kwargs)>

<Figure size 1296x864 with 0 Axes>

Part-of-speech (POS) tagging is a natural language processing (NLP) task that involves assigning grammatical tags to each word in a given text. The goal of POS tagging is to determine the syntactic category of each word, such as noun, verb, adjective, adverb, etc. These tags provide information about the role and function of words in a sentence, enabling further analysis and understanding of the text.

Hidden Markov Model (HMM) is a statistical model widely used in various fields, including NLP. HMM is a probabilistic model that assumes the underlying states of a system are hidden and can only be observed through a series of observable events or outputs. In the context of POS tagging, HMM is used to infer the most likely sequence of POS tags given an input sentence.

In POS tagging using HMM, the model considers a sequence of words as the input and assigns a sequence of POS tags as the output. The HMM assumes that there is a hidden Markov chain of POS tags, where each POS tag depends only on its previous POS tag. The observed words are emitted from each state (POS tag) based on their respective probabilities.

The training process involves estimating the transition probabilities between POS tags and the emission probabilities of words given a POS tag from a labeled corpus. These probabilities are then used to determine the most likely sequence of POS tags for unseen sentences.

During the tagging phase, given an input sentence, the HMM calculates the probability of different sequences of POS tags and selects the most likely sequence based on the Viterbi algorithm or other decoding techniques. This process enables the assignment of POS tags to words in the input sentence.

POS tagging with HMM is a common and effective approach for assigning grammatical tags to words in a given text, allowing for various downstream NLP tasks such as parsing, information extraction, and machine translation.